In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import glob
import cv2
import shutil  # <-- needed for copying

from utils import decompress_video, make_video, load_times

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from datetime import datetime, timezone, timedelta

#############################################
#############################################
#############################################
n_cams = 18
root_dir = "/home/cat/Downloads/data_stitching/cams/"
date = "2025_07_31"                               # date of the video
hour_start = "07"                                     # start at midnight
n_mins = 60
shrink_factor = 80   # this shrinks the video along x and y axis by this factor; for now we use subsampling
cage_id = 1

# outer loop gooin over every minue of time 
for minute in range(n_mins):

    # save the combined frame as a video file
    fname_combined = os.path.join(root_dir, "minute_" +str(minute)+'.avi')

    if os.path.exists(fname_combined):
        print ("Video exists for Minute ", str(minute), "...skipping...")
        print ('')
        print ('')
        continue

    # find the filename for each camera that falls in minute #1
    for cam in range(1,n_cams+1,1):
        
        #################################################
        ############## Generate Metadata ################
        #################################################
        
        fname_root = os.path.join(root_dir, str(cam), 
                                  str(cage_id) + "_" +
                                  str(cam) + "_" +
                                  date+"_"+str(hour_start)+"_"+str(minute).zfill(2)+"*.npz")

        #print ("fname root: ", fname_root)
        # find any filenmes that match this pattern
        # There can only be 0 or 1 videos at most as we save 1 minute video per camera
        fnames = glob.glob(fname_root)

        # if the camera has files we then need to load the frame times
        if len(fnames)==0:
            print ('... no files found for camera: ', cam, " minute: ", minute)
            print ('')
            continue           
        print ("minute:", minute, " cam:", cam ) #, " files:", fnames)

        # load the time stamps for this camera
        time_stamps_binned = load_times(fnames[0], minute)
        print ("time stamps binned: ", time_stamps_binned)

        # Convert to UTC+1 using timezone offset
        dt_naive = datetime.strptime(f"{date.replace('_', '-')} {hour_start}:{minute:02d}", "%Y-%m-%d %H:%M")
        dt_utc1 = dt_naive.replace(tzinfo=timezone(timedelta(hours=1)))  # UTC+1

        # Get absolute time in nanoseconds
        timestamp_ns = int(dt_utc1.timestamp() * 1_000_000_000)

        # Truncate to milliseconds
        unix_time_to_start_of_minute = timestamp_ns // 1_000_000
        print("absolute unix time (ms, UTC+1) to start of minute:", unix_time_to_start_of_minute)

        #################################################
        ############### Decompress video ################
        #################################################
        # process #1
        fname_video = fnames[0].replace('_metadata.npz', '.h264') 
        # use opencv to uncomrpess the video to .png files on disk
        decompress_video(minute,
                         fname_video,
                         root_dir,
                         cam,
                         time_stamps_binned,
                         unix_time_to_start_of_minute, 
                         shrink_factor,
                         overwrite_existing=True)     
        print ('')

    #################################################
    ############### Make video ######################
    #################################################
    # process #2 - here we make the mosaic 1 minute video based on the available files
    # we loop over all possible files 

    make_video(root_dir,
               minute,
               n_cams,
               fname_combined,
               shrink_factor=shrink_factor,
               overwrite_existing=True)
    
    print ("***************************")
    print ('')

    

Video exists for Minute  0 ...skipping...


Video exists for Minute  1 ...skipping...


Video exists for Minute  2 ...skipping...


Video exists for Minute  3 ...skipping...


... no files found for camera:  1  minute:  4

minute: 4  cam: 2
time stamps binned:  [1753941895660 1753941895670 1753941895680 ... 1753941955660 1753941955660
 1753941955670]
absolute unix time (ms, UTC+1) to start of minute: 1753941840000
shrinking video frames to:  9 x 16
time stamps relative to start of minute:  [ 55660  55670  55680 ... 115660 115660 115670]
 total frames:  7205 total unique frames: 6002
File does not exist yet.
opening video file for reading:  /home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_04_55_681.h264
... clean exit, renaming current minute temp file to clean file ...
# UNIQUE frames written current min 434 , n_frames_read:  520
last frame time written:  60000  ctr_frame:  520
writing second minute frames starting at:  [10 10 20 30 40]
# UNIQUE frames written to next min vi

In [ ]:
fname = '/home/cat/Downloads/data_stitching/1/minute_1.bin'

# let's load this using mmap as a binary file
data = np.memmap(fname, dtype=np.uint8, mode='r', shape=(6000, 720, 1280, 3))

# let's check the shape of the data
print("Data shape:", data.shape)

# ok now let's grab a frame and display it
idx = 2000
frame = data[idx]

# let's display the frame using opencv
cv2.imshow("Frame", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


Data shape: (6000, 720, 1280, 3)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/cat/miniconda3/lib/python3.11/site-packages/cv2/qt/plugins"


In [ ]:
# Ignore - past here

In [110]:
fname0 = '/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_00_41_557_metadata.npz'
fname2 = '/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_01_44_905_metadata.npz'

frame_times1 = np.load(fname1)['frame_times']
frame_times2 = np.load(fname2)['frame_times']
print ((frame_times1 - 1753941704894859)//1000+49880, "msecs")
print ((frame_times2 - 1753941704894859)//1000+49880-60000, "msecs")


[ 49880  49888  49896 ... 109864 109872 109881] msecs
[-10120 -10112 -10104 ...  49864  49872  49881] msecs


In [109]:
fname_bin1 = '/home/cat/Downloads/data_stitching/cams/2/minute_1.bin'
data_bin1 = np.fromfile(fname_bin1).reshape(32,18,3)
# print size of file
print (data_bin1.size/32/18)

print (data_bin1.shape)

ValueError: cannot reshape array of size 897480 into shape (32,18,3)

In [ ]:


#########################################
# LOAD video time stamps
data = np.load(fnames_cam1[0], allow_pickle=True)
print (data.files)
frame_times_ms = data['frame_times']//1000
print (frame_times_ms)
recording_start_time = data['recording_start_time']
print ("recording start time: ", recording_start_time)
encoder_start = data['encoder_start']
print ("encoder start: ", encoder_start)

###############################################
# convert july 24, 2025  exacdtly midngith to milisecond in epoch systm eimte clock but make sure its' UTC+1 london time
epoch_start = np.datetime64('2025-07-24T00:00:00', 'ms') - np.timedelta64(1, 'h')  # UTC+1
epoch_start_ms = epoch_start.astype('datetime64[ms]').astype(int)

# so this is the video time stamps relative to the epoch start
delta_times = frame_times_ms - epoch_start_ms
print("time srelative to midnight (in ms): ", delta_times)

# now convert into bucket discrete time
delta_times_bucket = delta_times // 10 * 10 # convert to seconds
print("time relative to midnight in 10ms buckets: ", delta_times_bucket)

# and convert into a discrete bin of 10ms from midnight
delta_times_bucket_discrete = delta_times_bucket // 10
print("time relative to midnight in 10ms discrete buckets: ", delta_times_bucket_discrete)
print ('')
print (' So we either ruse this bueckt version which tells you which 10ms bucket the frame is in')
print (' or the disrete version which tells you exactly what frame of the video to put the uncompressed video data into')

In [51]:
#####################################
# make some fake video data but don't generate the full array as it will take 100GB of ram
data_cam1_vid1 = np.zeros((frame_times_ms.shape[0]))
print ("video data (ordinarlily this would be an .mp4 that's decompressed on the fly: ", data_cam1_vid1.shape)

# and we crate a 1 min video bucket to hold the videos
stitched_video = np.zeros((18, 60000))
print ("stitched video (18 rpis, 60000 times should be about 10minutes (100fps * 60 * 10)): ", stitched_video.shape)
print (" (the full video is much larger as it has the frames 1024 , 768 and 3 channels for RGB)")

# so now we can loop over the relative to midnight time dicscrete buickets
for i in range(delta_times_bucket_discrete.shape[0]):
    bucket = delta_times_bucket_discrete[i]
    if i%1000==0:
        print("processing frame ", i, " in bucket ", bucket)
    
    # so now we can put the video data into the stitched video
    # this is just a fake example, in reality you would decompress the video and put it into the stitched video
    stitched_video[0, bucket] = data_cam1_vid1[i]  # assuming we are putting cam1 data into the first row


video data (ordinarlily this would be an .mp4 that's decompressed on the fly:  (7205,)
stitched video (18 rpis, 60000 times should be about 10minutes (100fps * 60 * 10)):  (18, 60000)
 (the full video is much larger as it has the frames 1024 , 768 and 3 channels for RGB)
processing frame  0  in bucket  7589
processing frame  1000  in bucket  8422
processing frame  2000  in bucket  9255
processing frame  3000  in bucket  10088
processing frame  4000  in bucket  10921
processing frame  5000  in bucket  11754
processing frame  6000  in bucket  12587
processing frame  7000  in bucket  13420


In [19]:
print("Files:", data.files)
print("First 5 frame_times_ms:", frame_times_ms[:5])
print("Recording start time:", recording_start_time)
print("Encoder start:", encoder_start)
print("Midnight reference:", epoch_start_ms)
print("First timestamp:", frame_times_ms[0])
print("Delta (first):", delta_times[0])


Files: ['frame_times', 'frames_encoded', 'recording_start', 'recording_end', 'encoder_start', 'frame_rate', 'recording_duration', 'recording_mode', 'recording_type', 'recording_duration_continuous', 'recording_duration_triggered', 'recording_start_time']
First 5 frame_times_ms: [1753311675898 1753311675906 1753311675914 1753311675923 1753311675931]
Recording start time: 2025-07-24_00-01-15
Encoder start: 1753311675.907574
Midnight reference: 1753315200000
First timestamp: 1753311675898
Delta (first): -3524102


In [20]:
print("Recording start time (converted):", 
      np.datetime64(int(recording_start_time), 'ms'))


ValueError: invalid literal for int() with base 10: '2025-07-24_00-01-15'